In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!git clone https://github.com/utkarsh512/CreateDebate-Scraper.git

In [ ]:
%cd CreateDebate-Scraper/src/nested/

In [ ]:
from thread import Thread, Comment
import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from tqdm import tqdm
import matplotlib
matplotlib.rcParams.update({'font.size': 18})

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
from   matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# sns.set(style='darkgrid')
# matplotlib.rcParams['figure.dpi'] = 120
matplotlib.rcParams['font.size'] = 18
# matplotlib.rcParams['figure.figsize'] = (10, 5)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
threads = list()

with open('/content/gdrive/MyDrive/DL/CreateDebate/politics2/threads.log', 'rb') as f:
    try:
        while True:
            threads.append(pickle.load(f))
    except EOFError:
        pass

In [ ]:
len(threads)

In [ ]:
v = list(threads[0].comments.items())[0][1].time
print(v)

In [ ]:
threadsByYear = dict()
errorCount = 0

for e in threads:
    try:
        threadTime = list(e.comments.items())[0][1].time
        if threadTime == 'Not Available':
            continue 
        threadTime = threadTime[:4]
        try:
            threadsByYear[threadTime].append(e)
        except:
            threadsByYear[threadTime] = list()
            threadsByYear[threadTime].append(e)
    except:
        assert len(list(e.comments.items())) == 0
        errorCount += 1
print(errorCount)

In [ ]:
threadsByYear.keys()

In [ ]:
threadsBefore = list()
threadsAfter = list()

In [ ]:
for key in ['2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008']:
    for e in threadsByYear[key]:
        threadsBefore.append(e)

for key in ['2021', '2020', '2019', '2018', '2017']:
    for e in threadsByYear[key]:
        threadsAfter.append(e)

In [ ]:
def build_graph(threads, n1 = 0, n2 = 0):
    """Builds support graph and dispute graph from hyper-parameters n1 and n2
    inputs
    :param n1: threshold on number of level-1 comments
    :param n2: threshold on number of direct replies

    output
    (author_map : dict, reverse_map : list, author_count : int, support_graph : nx.DiGraph, support_matrix: list, dispute_graph : nxDiGraph, dispute_matrix : list)
    """

    athr = dict()
    for e in threads:
        if 'root' in e.metaL.keys():
            for key in e.metaL['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1
        if 'root' in e.metaR.keys():
            for key in e.metaR['root'].keys():
                cmnt = e.comments[key]
                cur_athr = cmnt.author
                try:
                    athr[cur_athr] += 1
                except:
                    athr[cur_athr] = 1

    L1_athr = dict()
    for x in athr:
        if athr[x] >= n1:
            L1_athr[x] = True

    athr = dict()

    def dfs(Map, cmntMap, athr, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs(Map[cid], cmntMap, athr, key)
            return
        cur_author = cmntMap[cid].author

        try:
            athr[cur_author] += len(Map[cid].keys())
        except:
            athr[cur_author] = len(Map[cid].keys())

        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, athr, key)

    for e in threads:
        if 'root' in e.metaL.keys():
            dfs(e.metaL, e.comments, athr)
        if 'root' in e.metaR.keys():
            dfs(e.metaR, e.comments, athr) 

    A = []
    for x in athr:
        if athr[x] >= n2:
            try:
                z = L1_athr[x]
                A.append(x)
            except KeyError:
                pass

    author_map = dict()
    reverse_map = [""] * len(A)
    author_count = len(A)

    for i in range(author_count):
        author_map[A[i]] = i
        reverse_map[i] = A[i]

    support_matrix = [[0 for j in range(author_count)] for i in range(author_count)]
    dispute_matrix = [[0 for j in range(author_count)] for i in range(author_count)]

    def dfs1(Map, cmntMap, cid='root'):
        if cid == 'root':
            for key in Map[cid].keys():
                dfs1(Map[cid], cmntMap, key)
            return

        cur_author = cmntMap[cid].author
        cur_pol = cmntMap[cid].polarity
        
        if cur_author in author_map and cur_pol != 'Not Available':
            cur_author_id = author_map[cur_author]
            for key in Map[cid].keys():
                nxt_author = cmntMap[key].author
                nxt_pol = cmntMap[key].polarity
                if nxt_author in author_map and nxt_pol != 'Not Available':
                    nxt_author_id = author_map[nxt_author]
                    if cur_pol == nxt_pol:
                        support_matrix[nxt_author_id][cur_author_id] += 1
                    else:
                        dispute_matrix[nxt_author_id][cur_author_id] += 1

        for key in Map[cid].keys():
            dfs1(Map[cid], cmntMap, key)

    for e in threads:
        if 'root' in e.metaL:
            dfs1(e.metaL, e.comments)
        if 'root' in e.metaR:
            dfs1(e.metaR, e.comments)

    support_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if support_matrix[i][j] != 0:
                support_graph.add_weighted_edges_from([(i, j, support_matrix[i][j])])

    dispute_graph = nx.DiGraph()
    for i in range(author_count):
        for j in range(author_count):
            if dispute_matrix[i][j] != 0:
                dispute_graph.add_weighted_edges_from([(i, j, dispute_matrix[i][j])])
    
    return (author_map, reverse_map, author_count, support_graph, support_matrix, dispute_graph, dispute_matrix)

In [ ]:
thresholds = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
thresholds_str = []
for x in thresholds:
    thresholds_str.append(str(x))
n = len(thresholds)
count = [[0 for j in range(n)] for i in range(n)]
support_graph_r = [[0 for j in range(n)] for i in range(n)]
dispute_graph_r = [[0 for j in range(n)] for i in range(n)]
with tqdm(total=n*n) as pbar:
    for i in range(n):
        for j in range(n):
            _1, _2, cnt, support_graph, _4, dispute_graph, _6 = build_graph(threadsBefore, thresholds[i], thresholds[j])
            count[i][j] = cnt
            try:
                support_graph_r[i][j] = nx.algorithms.reciprocity(support_graph)
            except:
                support_graph_r[i][j] = 0
            try:
                dispute_graph_r[i][j] = nx.algorithms.reciprocity(dispute_graph)
            except:
                dispute_graph_r[i][j] = 0
            pbar.update(1)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(support_graph_r, interpolation='nearest', vmin=0, vmax=1)
fig.colorbar(cax)
ax.set_xticks(np.arange(n))
ax.set_yticks(np.arange(n))
ax.set_xticklabels(thresholds_str)
ax.set_yticklabels(thresholds_str)
ax.set_ylabel("$\lambda$", rotation='horizontal')
ax.set_xlabel("$\\rho$")
plt.setp(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
temp = [[support_graph_r[i][j] for j in range(9)] for i in range(8)] #x, y

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(temp, cmap="crest", ax=ax, vmin=0.5, vmax=0.9)
ax.set_xticks(np.arange(9))
ax.set_yticks(np.arange(8))
ax.set_xticklabels(thresholds_str[:9])
ax.set_yticklabels(thresholds_str[:8])
ax.set_ylabel("$\lambda$", rotation='horizontal')
ax.set_xlabel("$\\rho$")
plt.setp(ax.get_xticklabels(), rotation=90)
plt.setp(ax.get_yticklabels(), rotation=0)
fig.savefig('temporal_analysis_support_graph_reciprocity_before_US_debates.pdf', format='pdf', bbox_inches="tight")

In [ ]:
temp = [[dispute_graph_r[i][j] for j in range(9)] for i in range(8)] #x, y

In [ ]:
fig, ax = plt.subplots()
sns.heatmap(temp, cmap="crest", ax=ax, vmin=0.5, vmax=0.9)
ax.set_xticks(np.arange(9))
ax.set_yticks(np.arange(8))
ax.set_xticklabels(thresholds_str[:9])
ax.set_yticklabels(thresholds_str[:8])
ax.set_ylabel("$\lambda$", rotation='horizontal')
ax.set_xlabel("$\\rho$")
plt.setp(ax.get_xticklabels(), rotation=90)
plt.setp(ax.get_yticklabels(), rotation=0)
fig.savefig('temporal_analysis_dispute_graph_reciprocity_before_US_debates.pdf', format='pdf', bbox_inches="tight")

In [ ]:
author_map, reverse_map, author_count, support_graph, support_matrix, dispute_graph, dispute_matrix = build_graph(threads, 0, 0)

In [ ]:
author_count

In [ ]:
threads_ = list()
new_topic_ = 'technology'

with open(f'/content/gdrive/MyDrive/DL/CreateDebate/{new_topic_}/threads.log', 'rb') as f:
    try:
        while True:
            threads_.append(pickle.load(f))
    except EOFError:
        pass

In [ ]:
author_map1, reverse_map1, author_count1, support_graph1, support_matrix1, dispute_graph1, dispute_matrix1 = build_graph(threads_,0, 0)

In [ ]:
author_count1

In [ ]:
inter = set(reverse_map) & set(reverse_map1)

print(len(inter) / len(reverse_map1))

In [ ]:
reverse_map1